# Decision Tree

决策树的难点在于如何决定对属性进行划分的顺序, 比如下面的例子, root node 应该选哪个属性(色泽?根蒂?...)呢? 对于这个问题决策树给出了最为通用的三种算法:

- 信息增益(ID3)
- 信息增益率(C4.5)
- 基尼系数(CART)

下面对这三种算法做详细的介绍.

为了更好的理解算法, 我们设计了一份数据, 该数据给出了西瓜的诸多属性, 并提供了是"好瓜"还是"坏瓜"的标签. 我们将利用 决策树 模型来进行建模.

首先我们使用 pandas 库, 导入样例数据

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlt

libs = [('numpy', np), ('pandas', pd), ('matplotlib', mlt)]
print()
for lib in libs:
    print(f'{lib[0]:>10} version is {lib[1].__version__}')


     numpy version is 1.17.2
    pandas version is 0.25.1
matplotlib version is 3.1.1


In [3]:
df = pd.read_excel('../data/choice_watermelon.xlsx')
df

,编号,色泽,根蒂,敲声,纹理,脐部,触感,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,是
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,否
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,否


## 信息熵(Information Entropy)

因为后面的公式都会使用信息熵, 因此这里先介绍一下信息熵的定义.

集合 $D$ 中第 $k$ 类样本所占的比率为 $p_{k} (k={1,2,...,|n|})$

则 $D$ 的信息熵为: $Ent(D) = -\displaystyle\sum_{k=1}^{n} p_{k}\log_{2}p_{k}$

e.g. 上面样例集合 $D$ 的分类目标是属性 "好瓜", 该属性有两类值:"是"和"否", 因此 $n=2$ $k=1$表示"是", $k=2$表示"否", $p_1$表示"是"的概率, $p_2$表示"否"的概率, 套入信息熵公式为:

$
\begin{align*}
    Ent(D) &= -\displaystyle\sum_{k=1}^{n} p_{k}\log_{2}p_{k} \\
           &= -\displaystyle\sum_{k=1}^{2} p_{k}\log_{2}p_{k} \\
           &= -p_{1}\log_{2}p_{1} - p_{2}\log_{2}p_{2} \\
           &= -\frac{8}{17}\log_{2}\frac{8}{17} - \frac{9}{17}\log_{2}\frac{9}{17} \\
           &= 0.998
\end{align*}
$

In [4]:
# 计算信息熵
ent_D = -8/17 * np.log2(8/17) - 9/17 * np.log2(9/17)
print(f'信息熵 D 的值为: {ent_D}')

信息熵 D 的值为: 0.9975025463691153


## 信息增益(Information Gain) - ID3

Information Gain 的数据公式表示为:

$
Gain(D, a) = Ent(D) - \displaystyle \sum_{v=1}^{V} \frac{|D^{v}|}{|D|} Ent(D^{v})
$

- $D$: 样本集合
- $a$: 样本集合中的某一离散属性 (e.g. 色泽)
- $Gain(D, a)$: 属性 $a$ 的信息增益(information gain)
- $Ent(D)$: 集合 $D$ 的信息熵(information entropy)
- $V$: 属性 $a$ 有 $V$ 个可能的取值 $\{a^1, a^2, ..., a^V\}$
- $v$: 属性 $a$ 对样本集 $D$ 进行划分, 会有 $V$ 个分支. 其中第 $v$ 个分支节点包含了样本集合 $D$ 中所有在属性 $a$ 上取值为 $a^v$ 的样本，记作 $D^v$
- $\frac{|D^{v}|}{|D|}$: 在属性 $a$ 上第 $v$ 个分支的权重
- $Ent(D^{v})$: 在属性 $a$ 上第 $v$ 个分支的信息熵

信息增益越大，意味着使用属性 $a$ 对样本集合 $D$ 进行划分所获得的 “纯度提升”越大, 用数学公式表示为:

$
a_* = \underset{a \in A}{\arg\max} Gain(D, a)
$

根据信息增益理论, 将其应用到我们的样例中, 属性 $A = \text{\{'编号', '色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '好瓜'\}}$, 因为 '编号' 没有实际的意义, 我们暂且不与考虑, 因此 属性 $A = \{a_1='色泽', \;a_2='根蒂', \;a_3='敲声', \;a_4='纹理', \;a_5='脐部', \;a_6='触感', \;a_7='好瓜'\}$ 共 7 种. 在这 7 种属性中, 应该选择哪个属性最为决策树的根呢? 这就需要我们分别求出这 7 个属性的信息增益, 其中信息增益最大的那个就可以作为决策树的根属性.

以属性 '色泽' 为例, 也就是需要求出 $Gain(D, a_1)$ 或 $Gain(D, '色泽')$.

我们先看看一下 '色泽' 属性的数据情况

In [6]:
# get series
color_lustre = df['色泽']
# 除重操作
color_lustre.drop_duplicates()

0    青绿
1    乌黑
4    浅白
Name: 色泽, dtype: object

In [7]:
df.groupby(['色泽', '好瓜'])['编号'].count()

色泽  好瓜
乌黑  否     2
    是     4
浅白  否     4
    是     1
青绿  否     3
    是     3
Name: 编号, dtype: int64

从上面的分析得出, 属性 '色泽' 去重后有三个值, 那么我们可以这么设定, $a_1='色泽'$, $a_1 = \{a_1^1='青绿', a_1^2='乌黑', a_1^3='浅白'\}$.

这样我们就可以将样本集划分为三个子集

- $D^1(色泽=青绿)$
- $D^2(色泽=乌黑)$
- $D^3(色泽=浅白)$

现在我们分别求上述三个子集的信息熵:

$
\begin{align*}
    Ent(D^1) &= -\displaystyle\sum_{k=1}^{n}p_{k}\log_2p_{k} \\
             &= -\displaystyle\sum_{k=1}^{2} p_{k}\log_{2}p_{k} \\
             &= -p_{1}\log_{2}p_{1} - p_{2}\log_{2}p_{2} \\
             &= -\frac{3}{6}\log_{2}\frac{3}{6} - \frac{3}{6}\log_{2}\frac{3}{6} \\
             &= 1
\end{align*}
$

$
\begin{align*}
    Ent(D^2) &= -\displaystyle\sum_{k=1}^{n}p_{k}\log_2p_{k} \\
             &= -\displaystyle\sum_{k=1}^{2} p_{k}\log_{2}p_{k} \\
             &= -p_{1}\log_{2}p_{1} - p_{2}\log_{2}p_{2} \\
             &= -\frac{4}{6}\log_{2}\frac{4}{6} - \frac{2}{6}\log_{2}\frac{2}{6} \\
             &= 0.918
\end{align*}
$

$
\begin{align*}
    Ent(D^3) &= -\displaystyle\sum_{k=1}^{n}p_{k}\log_2p_{k} \\
             &= -\displaystyle\sum_{k=1}^{2} p_{k}\log_{2}p_{k} \\
             &= -p_{1}\log_{2}p_{1} - p_{2}\log_{2}p_{2} \\
             &= -\frac{1}{5}\log_{2}\frac{1}{5} - \frac{4}{5}\log_{2}\frac{4}{5} \\
             &= 0.722
\end{align*}
$

现在我们就可以求 $a_1=色泽$ 的信息熵:

$
\begin{align*}
    Gain(D, a_1) &= Ent(D) - \displaystyle \sum_{v=1}^{V} \frac{|D^{v}|}{|D|} Ent(D^{v}) \\
                 &= 0.998 - (\frac{6}{17} \times 1 + \frac{6}{17} \times 0.918 + \frac{5}{17} \times 0.722) \\
                 &= 0.1
\end{align*}
$

下面我们使用 python 实现信息熵和信息增益